In [1]:
import os
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM

In [2]:
mpl8_dir = 'local/'
drp = fits.open(mpl8_dir + 'drpall-v2_5_3.fits')
drpdata = drp[1].data 
stellar_mass = drpdata.field('nsa_sersic_mass')

In [3]:
# getting MPA-JHU catalog information
gal = fits.open(mpl8_dir + 'gal_info_dr7_v5_2.fit')
galdata = gal[1].data
sfr = fits.open(mpl8_dir + 'gal_totsfr_dr7_v5_2.fits')
sfrdata = sfr[1].data
mass = fits.open(mpl8_dir + 'gal_totsfr_dr7_v5_2.fits')
massdata = mass[1].data

In [4]:
# getting information from NSA catalog
nsa = fits.open(mpl8_dir + '1-nsa_v1_0_1.fits')
nsa_data = nsa[1].data

In [5]:
# matching the MPA-JHU and MaNGA catalogs on RA and DEC
manga_ra = drpdata.field('objra')
manga_dec = drpdata.field('objdec')
mpa_ra = galdata.field('RA')
mpa_dec = galdata.field('DEC')
bad = mpa_dec < -90
mpa_dec[bad] = -90
mpa_cat = SkyCoord(ra=mpa_ra*u.degree, dec=mpa_dec*u.degree)
c = SkyCoord(ra=manga_ra*u.degree, dec=manga_dec*u.degree)
idx_mpa, d2d_mpa, d3d_mpa = c.match_to_catalog_sky(mpa_cat) 

In [32]:
sfr_manga = sfrdata.field('MEDIAN')[idx_mpa]
re_arcsec = drpdata.field('nsa_elpetro_th50_r')#nsa_data.field('ELPETRO_TH50_R')
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
z = drpdata.field('nsa_z')
print(len(re_arcsec))
len(z)
#drpdata

6779


6779

In [7]:
re_kpc = re_arcsec * u.arcsec / cosmo.arcsec_per_kpc_proper(z)
sigma_sfr = 10**sfr_manga / (2 * np.pi * re_kpc**2)

/Users/rileytheriault/anaconda3/lib/python3.7/site-packages/astropy/cosmology/core.py:1922: RuntimeWarning: invalid value encountered in sqrt
  return 2 * np.sqrt(x) * hyp2f1(1./6, 1./2, 7./6, -x**3)


In [8]:
print(re_kpc)

[ 4.01112287  5.65680643  5.91935481 ...  4.21399515 10.6887845
  3.49211062] kpc


In [9]:
print(sigma_sfr)
sort = np.argsort(sigma_sfr)
print(sort[6300:6400])
print(sigma_sfr[sort][6300:6400])

[0.00936724 0.00145146 0.00743165 ... 0.02340948 0.00030319 0.01142684] 1 / kpc2
[6450 3402 5837 3403 6001 5242 4891 4902 3365 6106 3033  176 5058  934
  976 3118 1447 3742  910 6650 4831 3851 2284 6395 4939 4085 4173 4553
  967 5937 6484 5170 4899 5123 2969 4886 5737 4804 5680 3170 5363 1774
  302 5938 5868 2616 6734 2639 6764 6771 6311 2278 1524 3819 1957  689
  213 1838 1855  810 1656 5431 2745 6029 4915 5631 4141  211 3732 4746
 6690 4642  348 4798 4850 6683 1261 4184 2928 5341 1674 3337 1809 1231
 2671 4392 6020 3202 1602 3578 5308 4475 1176 1009 3557 3058  231 5179
 5192 1193]
[0.06605533 0.06735494 0.06750741 0.06773563 0.06774775 0.06784296
 0.06871549 0.06872746 0.06887275 0.06897282 0.06967202 0.07001213
 0.0701695  0.070444   0.07084638 0.0711444  0.07121488 0.07146561
 0.07174506 0.07236975 0.0726841  0.07310981 0.07361492 0.07378026
 0.07390901 0.07403148 0.07543778 0.07602351 0.07646497 0.07664408
 0.07754349 0.07782598 0.07797624 0.0781509  0.07967443 0.0797261
 0.080186

In [10]:
drpdata.field('plateifu')[sort][6300:6400]

chararray(['9875-6101', '8615-1901', '9497-3703', '8615-1902',
           '9507-12704', '9086-3703', '9032-9101', '9033-3703',
           '8613-12703', '9513-1902', '8553-3704', '10215-1902',
           '9042-6102', '7993-9101', '8079-1902', '8568-3704',
           '8153-1901', '8719-1901', '7992-3702', '9887-3701',
           '9029-1902', '8725-6102', '8335-3703', '9872-3701',
           '9035-6102', '8941-3702', '8946-6101', '8992-12701',
           '8078-6104', '9503-3701', '9877-6101', '9049-1901',
           '9033-1902', '9046-3703', '8549-6104', '9032-3704',
           '9491-6101', '9027-6103', '9488-1901', '8592-6101',
           '9094-6101', '8257-3703', '10492-6103', '9503-3702',
           '9499-1902', '8454-12702', '9892-1902', '8455-3701',
           '9894-12703', '9894-3703', '9867-3702', '8335-12704',
           '8158-6104', '8723-6104', '8313-1901', '7815-3702',
           '10217-3703', '8261-1901', '8262-1901', '7972-3704',
           '8249-3704', '9183-6101', '8464-610

In [11]:
sigma_sfr_nonan = sigma_sfr[np.logical_not(np.isnan(sigma_sfr))]
ten = np.percentile(sigma_sfr_nonan,10)
print(ten)
ninety = np.percentile(sigma_sfr_nonan,90)
print(ninety)
gal_high_sigma_sfr = (sigma_sfr*u.kpc*u.kpc > ninety)
gal_low = (sigma_sfr*u.kpc*u.kpc < ten)
#print(sigma_sfr[gal_high_sigma_sfr])
print(np.median(sigma_sfr[gal_high_sigma_sfr]))
print(np.median(sigma_sfr[gal_low]))
drpdata.field('plateifu')[gal_high_sigma_sfr]

0.0002770038881975685
0.022082837804183623
0.0411884818785878 1 / kpc2
0.00013897655317165995 1 / kpc2


/Users/rileytheriault/anaconda3/lib/python3.7/site-packages/astropy/units/quantity.py:463: RuntimeWarning: invalid value encountered in greater
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/rileytheriault/anaconda3/lib/python3.7/site-packages/astropy/units/quantity.py:463: RuntimeWarning: invalid value encountered in less
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


chararray(['10001-3702', '10001-3703', '10214-1902', '10215-1902',
           '10215-3704', '10215-6103', '10215-6104', '10216-6101',
           '10216-6103', '10217-3701', '10217-3703', '10217-6103',
           '10218-1901', '10218-1902', '10218-3704', '10218-6102',
           '10218-6103', '10219-12702', '10220-3703', '10221-3704',
           '10221-6102', '10492-6103', '10492-9101', '10493-3704',
           '10494-3701', '10496-12704', '10496-3702', '10497-6101',
           '10498-3704', '10499-3702', '10503-3702', '10504-12701',
           '10504-9101', '10506-1901', '10507-1901', '10508-1901',
           '10509-12703', '10509-1901', '10509-3703', '10509-3704',
           '10509-6101', '10512-1902', '10516-12701', '10516-1902',
           '10516-3702', '10517-3701', '10517-3704', '10517-6102',
           '7443-12703', '7443-3703', '7495-3701', '7815-3702',
           '7815-6103', '7958-6101', '7962-6104', '7962-9101',
           '7968-3704', '7972-3704', '7977-12701', '7977-3703',


In [12]:
ba = drpdata.field('nsa_sersic_ba')

# get things to make that c_manga thing
nsa_ra = nsa_data.field('RA')
nsa_dec = nsa_data.field('DEC')
catalog = SkyCoord(ra=nsa_ra*u.degree, dec=nsa_dec*u.degree)
idx, d2d, d3d = c.match_to_catalog_sky(catalog) 

th50 = nsa_data.field('ELPETRO_TH50_R')
th90 = nsa_data.field('ELPETRO_TH90_R')
th50_manga = th50[idx]
th90_manga = th90[idx]
c_manga = th90_manga / th50_manga

# I think this one is good?

In [13]:
# then define things that are late type (concentration) then edge-on (axis ratio)
blah = drpdata.field('srvymode') == 'MaNGA dither'
print(len(c_manga[blah]))

# getting the gals
c_manga_gal = c_manga[blah]
ba_gal = ba[blah]

#getting the late-type ones
late = c_manga_gal < 2.6

ba_gal_late = ba_gal[late]
c_manga_gal_late = c_manga_gal[late]

print(len(c_manga_gal_late))

good_late = np.logical_not(np.isnan(c_manga_gal_late))
print(len(good_late))
# ok so after that business then do what we did about to get the ten percent of the late type, edge-on galaxies and I 
# should ghet about sixty of those. After that, make a slideshow like the one I did earlier with my special galaxies 
# to compare their maps. neato.

6661
3033
3033


In [14]:
edge = ba_gal_late < 0.3

# update concentration and ba arrays
c_manga_gal_late_edge = c_manga_gal_late[edge]
ba_gal_late_edge = ba_gal_late[edge]

print(len(c_manga_gal_late_edge))

647


In [15]:
sigma_sfr_us = sigma_sfr[blah][late][edge]
good = np.logical_not(np.isnan(sigma_sfr_us))
#print(sigma_sfr_us)
#sigma_sfr_awesome = sigma_sfr_us[np.logical_not(np.isnan(sigma_sfr_us))]
sigma_sfr_awesome = sigma_sfr_us[good]
print(len(sigma_sfr_us), len(sigma_sfr_awesome))

647 596


In [16]:
ten = np.percentile(sigma_sfr_awesome,10)#c_manga_gal_late_edge,10)
print(ten)
ninety = np.percentile(sigma_sfr_awesome,90)#c_manga_gal_late_edge,90)
print(ninety)
gal_high_c_manga_gal_late_edge = (sigma_sfr_awesome*u.kpc*u.kpc > ninety)
#print(c_manga_gal_late_edge[gal_high_c_manga_gal_late_edge])
#print(gal_high_c_manga_gal_late_edge)
plateifu_high = drpdata.field('plateifu')[blah][late][edge][good][gal_high_c_manga_gal_late_edge]
print(plateifu_high)
print(len(plateifu_high))
sigma_sfr_high = sigma_sfr_awesome[gal_high_c_manga_gal_late_edge]
print(sigma_sfr_high)

0.0005706972328089633
0.011793797735198196
['10220-6104' '7960-12702' '7960-6103' '7979-12701' '8077-9101'
 '8081-1901' '8083-3703' '8085-3701' '8086-3704' '8137-3704' '8138-6101'
 '8138-6103' '8141-3701' '8144-12705' '8249-1901' '8252-9101' '8260-6101'
 '8323-6103' '8324-12701' '8324-6103' '8325-3701' '8331-3703' '8332-3704'
 '8332-9102' '8439-6103' '8442-12701' '8453-1902' '8458-3701' '8465-3703'
 '8465-9101' '8547-9102' '8551-6104' '8600-6104' '8601-6103' '8603-6103'
 '8624-12702' '8711-1902' '8716-6102' '8939-6101' '8943-1901' '8950-1901'
 '8981-3703' '8992-12701' '8993-6104' '8999-3704' '9031-12704' '9035-3702'
 '9047-12705' '9086-6102' '9089-3704' '9090-1902' '9091-12704' '9184-3704'
 '9185-3704' '9190-3703' '9192-1901' '9193-9102' '9485-3701' '9891-6102'
 '9894-12703']
60
[0.02087265 0.02184589 0.01670782 0.01322442 0.01228151 0.11401257
 0.01677868 0.01476489 0.02052567 0.01302549 0.01659246 0.0153078
 0.01401969 0.01199777 0.04961361 0.01602484 0.02819373 0.03314391
 0.0337286

In [17]:
ten = np.percentile(sigma_sfr_awesome,10)
print(ten)
gal_high_c_manga_gal_late_edge_10 = (sigma_sfr_awesome*u.kpc*u.kpc < ten)
plateifu_10 = drpdata.field('plateifu')[blah][late][edge][good][gal_high_c_manga_gal_late_edge_10]
print(plateifu_10)
print(len(plateifu_10))
sigma_sfr_high_10 = sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_10]
print(sigma_sfr_high_10)

0.0005706972328089633
['10213-12703' '10503-12703' '10510-9101' '10512-9102' '10517-12701'
 '7958-12703' '7958-12704' '7977-12704' '7979-6102' '7990-12702'
 '7991-6102' '8078-12702' '8086-12702' '8097-12701' '8145-12705'
 '8149-12701' '8158-12701' '8244-12705' '8249-12702' '8249-9102'
 '8259-12702' '8260-12705' '8261-12704' '8312-6101' '8446-12703'
 '8450-12703' '8452-12702' '8464-12703' '8549-12704' '8597-6101'
 '8615-12702' '8618-12701' '8712-9102' '8713-3701' '8714-1901' '8931-6101'
 '8932-12702' '8935-12701' '8938-12705' '8945-12704' '8977-12704'
 '8988-12702' '8988-1901' '8994-12704' '9050-12703' '9086-12701'
 '9090-12703' '9181-12705' '9186-12703' '9491-12702' '9495-12705'
 '9496-12703' '9497-12702' '9499-12704' '9502-12704' '9502-12705'
 '9505-3703' '9870-12704' '9871-1902' '9885-12701']
60
[3.24287109e-04 8.07158552e-05 5.02229364e-04 4.59413370e-04
 3.21317444e-04 5.54518825e-04 2.10196939e-04 5.85942819e-05
 5.05359060e-04 3.64957612e-04 3.94645270e-04 1.47712114e-04
 2.17495

In [18]:
#trying to get greater than ten but less than twenty percentile of galaxies

ten = np.percentile(sigma_sfr_awesome,10)
print(ten)
twenty = np.percentile(sigma_sfr_awesome,20)
print(twenty)
gal_high_c_manga_gal_late_edge_20 = np.logical_and(sigma_sfr_awesome*u.kpc*u.kpc < twenty, 
                                                   sigma_sfr_awesome*u.kpc*u.kpc > ten) 
plateifu_20 = drpdata.field('plateifu')[blah][late][edge][good][gal_high_c_manga_gal_late_edge_20]
print(plateifu_20)
print(len(plateifu_20))
sigma_sfr_high_20 = sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_20]
print(sigma_sfr_high_20)

0.0005706972328089633
0.0010329571629079527
['10214-12701' '10493-12704' '10508-12704' '10514-12705' '7962-12705'
 '7964-12705' '7975-12703' '7979-12705' '7990-6102' '8077-12703'
 '8079-12701' '8081-12703' '8083-6101' '8139-6104' '8147-12705'
 '8152-12702' '8263-12702' '8317-6104' '8324-12705' '8330-9102'
 '8336-3702' '8446-6101' '8449-12703' '8449-6104' '8456-12705' '8484-6101'
 '8486-12703' '8486-12704' '8606-3704' '8655-12703' '8656-12701'
 '8714-12704' '8717-12704' '8717-12705' '8719-12704' '8727-1902'
 '8931-12703' '8932-3703' '8932-3704' '8934-6103' '8937-12703' '8937-6102'
 '8978-12701' '8997-12703' '9030-1901' '9034-3703' '9040-12703'
 '9048-12702' '9088-12703' '9190-12702' '9196-12704' '9494-6102'
 '9505-12702' '9869-12701' '9872-6103' '9872-6104' '9877-3703' '9886-1901'
 '9893-6102']
59
[0.00074065 0.00065022 0.00063958 0.00067327 0.00083046 0.00070716
 0.00085449 0.00096804 0.00060208 0.00091146 0.00102305 0.00089505
 0.00073752 0.00058049 0.00076054 0.00093931 0.00068452 0.

In [19]:
twenty

0.0010329571629079527

In [20]:
ten

0.0005706972328089633

In [21]:
#30th percentile

thirty = np.percentile(sigma_sfr_awesome,30)
print(twenty)
print(thirty)

gal_high_c_manga_gal_late_edge_30 = np.logical_and(sigma_sfr_awesome*u.kpc*u.kpc < thirty, 
                                                   sigma_sfr_awesome*u.kpc*u.kpc > twenty) 
plateifu_30 = drpdata.field('plateifu')[blah][late][edge][good][gal_high_c_manga_gal_late_edge_30]
print(plateifu_30)
print(len(plateifu_30))
sigma_sfr_high_30 = sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_30]
print(sigma_sfr_high_30)

0.0010329571629079527
0.0015304477575171689
['10215-9101' '10219-6103' '10221-3701' '10496-12703' '10498-12701'
 '7495-6104' '7958-12702' '7979-12704' '8097-3702' '8135-12705'
 '8141-12705' '8143-12703' '8148-6102' '8254-3703' '8259-12703'
 '8262-12705' '8323-6104' '8325-3703' '8326-12704' '8330-12704'
 '8337-12701' '8337-9101' '8338-12702' '8445-12704' '8446-12701'
 '8466-12701' '8550-3702' '8588-12704' '8588-6104' '8591-12703'
 '8602-6101' '8602-9102' '8612-12701' '8651-12701' '8727-12703'
 '8941-12704' '8978-12704' '8981-12704' '8986-12703' '8987-12704'
 '8987-12705' '8993-6101' '9024-9102' '9029-6104' '9034-6103' '9037-9102'
 '9184-12702' '9184-3701' '9188-12701' '9190-12705' '9195-12701'
 '9485-12702' '9500-3701' '9507-12701' '9866-12702' '9867-12701'
 '9867-9102' '9870-12705' '9885-6103']
59
[0.00150993 0.00149196 0.00126825 0.00114847 0.00151178 0.00109371
 0.00106634 0.00104118 0.00146069 0.0012545  0.00103339 0.00103461
 0.00134235 0.00145251 0.00117129 0.00109357 0.00117837 0

In [22]:
#40th percentile

forty = np.percentile(sigma_sfr_awesome,40)
print(thirty)
print(forty)

gal_high_c_manga_gal_late_edge_40 = np.logical_and(sigma_sfr_awesome*u.kpc*u.kpc < forty, 
                                                   sigma_sfr_awesome*u.kpc*u.kpc > thirty) 
plateifu_40 = drpdata.field('plateifu')[blah][late][edge][good][gal_high_c_manga_gal_late_edge_40]
print(plateifu_40)
print(len(plateifu_40))
sigma_sfr_high_40 = sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_40]
print(sigma_sfr_high_40)

0.0015304477575171689
0.00215752483929741
['10219-12705' '10493-12702' '10517-12703' '7957-6102' '7979-3703'
 '8143-12705' '8144-3704' '8144-9101' '8145-12701' '8150-12705'
 '8150-6103' '8239-6102' '8245-12701' '8248-12702' '8259-6102'
 '8309-12701' '8320-12702' '8329-9102' '8333-12703' '8455-6102'
 '8465-12702' '8568-6103' '8592-6104' '8600-3701' '8623-12702' '8716-3704'
 '8717-12701' '8724-12705' '8728-9101' '8934-1902' '8935-12704'
 '8947-12705' '8950-9101' '8981-9102' '8982-12701' '8990-12704'
 '8996-12702' '8996-12704' '8999-12705' '9032-3703' '9038-9101'
 '9047-9101' '9050-12701' '9087-12703' '9087-9102' '9184-12703'
 '9194-12702' '9195-12705' '9488-12704' '9491-6102' '9494-12701'
 '9496-1901' '9501-3703' '9509-12702' '9866-9102' '9885-12704' '9886-6102'
 '9889-6101' '9893-12704']
59
[0.00163948 0.00187629 0.00158543 0.00153806 0.00156415 0.0018459
 0.0016993  0.00205193 0.00153238 0.00179315 0.00172704 0.00185433
 0.0020074  0.00215274 0.00189816 0.00157697 0.00168123 0.00157023

In [23]:
#50th percentile

fifty = np.percentile(sigma_sfr_awesome,50)
print(forty)
print(fifty)

gal_high_c_manga_gal_late_edge_50 = np.logical_and(sigma_sfr_awesome*u.kpc*u.kpc < fifty, 
                                                   sigma_sfr_awesome*u.kpc*u.kpc > forty) 
plateifu_50 = drpdata.field('plateifu')[blah][late][edge][good][gal_high_c_manga_gal_late_edge_50]
print(plateifu_50)
print(len(plateifu_50))
sigma_sfr_high_50 = sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_50]
print(sigma_sfr_high_50)

0.00215752483929741
0.002846061991652449
['10497-3704' '10503-6102' '10511-12701' '7990-9102' '7992-3704'
 '8077-12702' '8080-9102' '8083-12701' '8139-12701' '8155-1901'
 '8158-12705' '8239-1902' '8253-12701' '8257-12702' '8257-12705'
 '8262-12701' '8309-12705' '8319-12705' '8324-6102' '8440-9102'
 '8445-12705' '8455-12705' '8461-12705' '8466-12703' '8547-12704'
 '8547-12705' '8548-12701' '8588-12701' '8655-12702' '8715-6103'
 '8720-12704' '8727-6104' '8940-12705' '8978-6102' '8982-12702'
 '8985-12704' '8987-6101' '8988-3701' '8996-12701' '9024-12702'
 '9037-12701' '9085-12702' '9085-9102' '9094-12701' '9094-12705'
 '9181-9102' '9189-6104' '9193-12702' '9486-12703' '9488-12701'
 '9491-12703' '9494-12702' '9494-12703' '9500-12703' '9510-12702'
 '9510-6101' '9869-12705' '9872-12702' '9872-12705']
59
[0.00273245 0.00279901 0.00225902 0.00217601 0.00283648 0.00283035
 0.0023061  0.00245141 0.00221426 0.002546   0.00270133 0.00219304
 0.00220088 0.00230968 0.00220316 0.00219333 0.00229593 0

In [24]:
#60th percentile

sixty = np.percentile(sigma_sfr_awesome,60)
print(fifty)
print(sixty)

gal_high_c_manga_gal_late_edge_60 = np.logical_and(sigma_sfr_awesome*u.kpc*u.kpc < sixty, 
                                                   sigma_sfr_awesome*u.kpc*u.kpc > fifty) 
plateifu_60 = drpdata.field('plateifu')[blah][late][edge][good][gal_high_c_manga_gal_late_edge_60]
print(plateifu_60)
print(len(plateifu_60))
sigma_sfr_high_60 = sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_60]
print(sigma_sfr_high_60)

0.002846061991652449
0.003579877393949376
['10217-12702' '10220-12702' '10496-6101' '10503-12704' '7443-12704'
 '7495-12702' '7990-12703' '8078-12704' '8081-12704' '8084-12705'
 '8139-12704' '8241-12703' '8247-12705' '8262-12702' '8311-12704'
 '8318-12702' '8320-12703' '8336-9101' '8337-12704' '8338-6103'
 '8444-12704' '8451-12705' '8456-12702' '8483-12701' '8483-3704'
 '8547-12702' '8550-9101' '8551-12704' '8555-12703' '8597-12705'
 '8615-9101' '8616-12705' '8619-12704' '8624-12705' '8655-6103'
 '8657-12703' '8713-12703' '8726-12703' '8937-12701' '8937-12704'
 '8941-6102' '8946-3702' '8947-12701' '8980-9102' '8987-3704' '9000-12705'
 '9001-6102' '9038-12704' '9038-9102' '9048-12703' '9190-9102'
 '9486-12704' '9496-12702' '9505-9102' '9506-9102' '9509-6101'
 '9867-12703' '9880-12705' '9893-9102']
59
[0.00318209 0.00294685 0.00300206 0.00296438 0.00304589 0.00306189
 0.00306898 0.00342984 0.00356516 0.00288035 0.00341293 0.00297631
 0.00317281 0.00348278 0.00308347 0.00287552 0.00314697

In [25]:
#70th percentile

seventy = np.percentile(sigma_sfr_awesome,70)
print(sixty)
print(seventy)

gal_high_c_manga_gal_late_edge_70 = np.logical_and(sigma_sfr_awesome*u.kpc*u.kpc < seventy, 
                                                   sigma_sfr_awesome*u.kpc*u.kpc > sixty) 
plateifu_70 = drpdata.field('plateifu')[blah][late][edge][good][gal_high_c_manga_gal_late_edge_70]
print(plateifu_70)
print(len(plateifu_70))
sigma_sfr_high_70 = sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_70]
print(sigma_sfr_high_70)

0.003579877393949376
0.0047007697669492424
['10220-12701' '10499-9102' '10507-12704' '10514-3701' '7964-12704'
 '8082-3702' '8084-12704' '8086-12703' '8137-12701' '8139-3703'
 '8143-9101' '8144-12702' '8156-12705' '8248-12701' '8250-12705'
 '8254-12702' '8258-12703' '8262-3704' '8263-12701' '8315-12704'
 '8317-6101' '8325-3702' '8446-12702' '8448-6102' '8465-12703'
 '8481-12702' '8481-12705' '8486-6104' '8595-3704' '8600-6102'
 '8612-12703' '8719-12705' '8719-6101' '8719-9102' '8727-12704'
 '8727-6102' '8934-1901' '8944-3701' '8946-12704' '8977-12701' '8983-6103'
 '8987-12703' '8992-3703' '9002-9102' '9032-9102' '9035-12704' '9035-9102'
 '9040-12704' '9086-12705' '9182-12705' '9188-12705' '9189-12701'
 '9494-12704' '9871-9101' '9884-12704' '9885-12705' '9889-12704'
 '9894-12705']
58
[0.00394279 0.00430458 0.00409394 0.0044822  0.00362202 0.00415847
 0.00379553 0.00402917 0.00385855 0.00406145 0.00426224 0.00394054
 0.00438618 0.00463883 0.003584   0.00359129 0.00456197 0.00422919
 0.00

In [26]:
#80th percentile

eighty = np.percentile(sigma_sfr_awesome,80)
print(seventy)
print(eighty)

gal_high_c_manga_gal_late_edge_80 = np.logical_and(sigma_sfr_awesome*u.kpc*u.kpc < eighty, 
                                                   sigma_sfr_awesome*u.kpc*u.kpc > seventy) 
plateifu_80 = drpdata.field('plateifu')[blah][late][edge][good][gal_high_c_manga_gal_late_edge_80]
print(plateifu_80)
print(len(plateifu_80))
sigma_sfr_high_80 = sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_80]
print(sigma_sfr_high_80)

0.0047007697669492424
0.007206463106729655
['10216-6104' '10496-12701' '10503-12702' '10503-3703' '10507-3702'
 '7495-3703' '7815-6101' '7972-3701' '7990-3702' '7990-9101' '7992-12703'
 '8077-6102' '8079-9102' '8137-12705' '8138-12705' '8149-12703'
 '8156-9102' '8249-12701' '8253-12704' '8257-9101' '8324-3702' '8324-6104'
 '8329-12703' '8337-9102' '8341-12701' '8445-9102' '8448-12704'
 '8462-9102' '8481-3701' '8484-12704' '8484-3704' '8485-3704' '8588-12705'
 '8612-12704' '8715-6102' '8716-12704' '8719-12701' '8942-1901'
 '8984-6101' '8986-12705' '8990-12701' '8993-6102' '8995-9101' '8996-6102'
 '8997-6103' '8998-6102' '9042-12701' '9088-12701' '9091-12703'
 '9091-6103' '9182-12702' '9184-6104' '9486-9102' '9498-12703' '9512-9101'
 '9514-12705' '9865-6101' '9871-3702' '9878-12703']
59
[0.00563357 0.00536412 0.00549476 0.00606636 0.0051004  0.00476894
 0.00574799 0.00574799 0.00642937 0.00496581 0.00472916 0.00636893
 0.00491844 0.00502441 0.00595227 0.00550667 0.00550469 0.00676958
 0.

In [27]:
other = np.percentile(sigma_sfr_awesome,80)
print(other)
print(ninety)

gal_high_c_manga_gal_late_edge_other = np.logical_and(sigma_sfr_awesome*u.kpc*u.kpc < ninety, 
                                                   sigma_sfr_awesome*u.kpc*u.kpc > eighty) 
plateifu_other = drpdata.field('plateifu')[blah][late][edge][good][gal_high_c_manga_gal_late_edge_other]
print(plateifu_other)
print(len(plateifu_other))
sigma_sfr_high_other = sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_other]
print(sigma_sfr_high_other)

0.007206463106729655
0.011793797735198196
['10001-12703' '10215-9102' '10506-12704' '10511-6101' '10519-12702'
 '7495-3702' '7975-3701' '7975-9102' '7991-6103' '7993-3701' '8078-9101'
 '8082-12703' '8131-12703' '8138-12702' '8140-12702' '8252-12703'
 '8255-12704' '8256-12701' '8274-12701' '8323-9102' '8328-12705'
 '8331-1902' '8334-12704' '8341-6103' '8440-3704' '8442-6102' '8443-9102'
 '8465-12704' '8485-6103' '8552-12702' '8611-6104' '8728-6103'
 '8933-12701' '8940-12703' '8942-6103' '8944-12702' '8944-12705'
 '8944-6101' '8945-9101' '8950-12701' '8950-12705' '8979-6101' '8980-9101'
 '8981-1902' '8988-12703' '8992-12702' '9033-3702' '9038-3701' '9091-9101'
 '9095-6104' '9493-3702' '9502-1901' '9505-6102' '9868-1902' '9878-12704'
 '9879-12703' '9884-12705' '9886-6104' '9887-12701']
59
[0.00743165 0.00998189 0.00975576 0.01043673 0.01019522 0.00780453
 0.00803605 0.01090241 0.00721075 0.00764816 0.01049172 0.00760253
 0.00817849 0.00973842 0.0101905  0.01150875 0.00819833 0.00829033
 0

In [31]:
onehunny = np.percentile(sigma_sfr_awesome,98)
print(onehunny)

0.03458990240569873


In [40]:
print(np.median(sigma_sfr[gal_low]))
print(np.median(sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_20]))
print(np.median(sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_30]))
print(np.median(sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_40]))
print(np.median(sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_50]))
print(np.median(sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_60]))
print(np.median(sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_70]))
print(np.median(sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_80]))
print(np.median(sigma_sfr_awesome[gal_high_c_manga_gal_late_edge_other]))
print(np.median(sigma_sfr[gal_high_sigma_sfr]))

0.00013897655317165995 1 / kpc2
0.0007605430838111735 1 / kpc2
0.0013157746160442961 1 / kpc2
0.00188096405485929 1 / kpc2
0.002471254986660658 1 / kpc2
0.0031036780037196616 1 / kpc2
0.004099177926695313 1 / kpc2
0.005633572111725045 1 / kpc2
0.008710864786317055 1 / kpc2
0.0411884818785878 1 / kpc2


In [ ]:
values = np.array([1.4e-4,7.6e-4,0.0013,0.0019,0.0025,0.0031,0.0041,0.0056,0.0087,0.041])